In [14]:
import geopandas as gpd

def convert_coordinates(df):
    # 좌표 변환
    geometry = gpd.points_from_xy(df['엑스좌표_값'], df['와이좌표_값'])
    gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:5181")
    gdf = gdf.to_crs("EPSG:4326")

    # 변환된 좌표 열 추가
    df['new_x'] = gdf.geometry.y
    df['new_y'] = gdf.geometry.x

    return df

In [15]:
import pandas as pd

commercial = pd.read_csv("data/commercial_with_xy.csv")

In [16]:
commercial = commercial[['상권_코드_명','총_생활인구_수','집객시설_수','점포_수','아파트_평균_면적','아파트_평균_시가','서울_운영_영업_개월_평균','서울_폐업_영업_개월_평균','매출_건수', "매출_금액",'구분', '엑스좌표_값', '와이좌표_값']]
commercial = commercial.drop_duplicates()
commercial.rename(columns={'서울_운영_영업_개월_평균': '영업_개월_평균', '서울_폐업_영업_개월_평균':'폐업_개월_평균','구분':'업종_명'},inplace=True)
commercial = commercial.reset_index(drop=True)

import numpy as np

commercial['아파트시가_log'] = np.log(commercial['아파트_평균_시가'])
commercial['매출_log'] = np.log(commercial['매출_금액'])
commercial.drop(["아파트_평균_시가", "매출_금액"], axis = 1, inplace = True)

In [17]:
commercial = convert_coordinates(commercial)

In [18]:
onehot = pd.get_dummies(commercial, columns = ['업종_명'])

In [19]:
onehot['업종_명_가전제품_및_통신기기_도소매업'] = 0
onehot['업종_명_개인_서비스업'] = 0
onehot['업종_명_교육_및_사업자원_서비스업'] = 0
onehot['업종_명_기타_도소매업'] = 0
onehot['업종_명_기타_서비스업'] = 0
onehot['업종_명_부동산_및_임대업'] = 0
onehot['업종_명_생활용품_도소매업'] = 0
onehot['업종_명_수리업'] = 0
onehot['업종_명_숙박업'] = 0
onehot['업종_명_스포츠_및_오락_서비스업'] = 0
onehot['업종_명_식료품_도소매업'] = 0
onehot['업종_명_음식점_및_주점업'] = 0
onehot['업종_명_제조업'] = 0
onehot['업종_명_중고상품_도소매업'] = 0

In [26]:
onehot.columns
X = onehot[["총_생활인구_수", "집객시설_수", "점포_수", "아파트_평균_면적", "아파트시가_log", "영업_개월_평균", "폐업_개월_평균", "매출_건수", '업종_명_가전제품_및_통신기기_도소매업',
       '업종_명_개인_서비스업', '업종_명_교육_및_사업자원_서비스업', '업종_명_기타_도소매업', '업종_명_기타_서비스업',
       '업종_명_부동산_및_임대업', '업종_명_생활용품_도소매업', '업종_명_수리업', '업종_명_숙박업',
       '업종_명_스포츠_및_오락_서비스업', '업종_명_식료품_도소매업', '업종_명_음식점_및_주점업', '업종_명_제조업',
       '업종_명_중고상품_도소매업', "매출_log", "new_x", "new_y"]]

In [25]:
X.to_csv("data/commercial_with_xy_epsg4326_onehot.csv", index=False)